## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import random

from funcoes import *
from funcoes import selecao_torneio_min as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento

In [2]:
from mofdb_client import fetch
# Fetch all mofs with void fraction >= 0.5 and <= 0.99
# Convert all isotherm loading units to mmol/g and all pressures to atmospheres

# Create a DataFrame
df = pd.DataFrame()

TAMANHO_DATAFRAME = 1000

t = 0
for mof in fetch(vf_min=0, vf_max=1, loading_unit="mmol/g", pressure_unit="atm"):
    new_row = {'mof': mof.name, 'cif': mof.cif, 'void fraction': mof.void_fraction}
    # Append the new row to the DataFrame
    df = df.append(new_row, ignore_index=True)
    t += 1
    if t >= TAMANHO_DATAFRAME:
        break

## Tratamento de dados

In [3]:
df = extrair_cif(df, minimo = 5)

c:\venv\ilumpy\lib\site-packages\pymatgen\io\cif.py:1134: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


GlobalSymmetryFeatures:   0%|          | 0/1000 [00:00<?, ?it/s]

ElementProperty:   0%|          | 0/1000 [00:00<?, ?it/s]

## Código e discussão



### Divisão treino-teste

In [4]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
FEATURES = ['spacegroup_num', # Em cristalografia, o grupo de espaço de um cristal é uma descrição da simetria do cristal. 1 - 230
            'crystal_system_int', # Sistema cristalino é a designação dada a um grupo de ordenamento espacial pontual regular de átomos ou moléculas. 1 - 7
            'n_symmetry_ops', # Sistema cristalino é a designação dada a um grupo de ordenamento espacial pontual regular de átomos ou moléculas. 1 - 12
            'Zn', # Quantidade de átomos de Zinco.
            'H', # Quantidade de átomos de Hidrogênio.
            'C', # Quantidade de átomos de Carbono.
            'O', # Quantidade de átomos de Oxigênio.
            'F', # Quantidade de átomos de Flúor.
            'Cl', # Quantidade de átomos de Cloro.
            'Br', # Quantidade de átomos de Bromo.
            'N' # Quantidade de átomos de Nitrogênio.
           ] 

TARGET = ['void fraction'] # porosidade do mof

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

In [5]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

In [6]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

In [7]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, neuronios_c3, neuronios_c4, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, neuronios_c3),
                    nn.ReLU(),
                    nn.Linear(neuronios_c3, neuronios_c4),
                    nn.ReLU(),
                    nn.Linear(neuronios_c4, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

In [8]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 150
NEURONIOS_C2 = 100
NEURONIOS_C3 = 80
NEURONIOS_C4 = 60

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NEURONIOS_C3, NEURONIOS_C4, NUM_DADOS_DE_SAIDA)

In [9]:
y_prev = minha_MLP(X_treino)

In [10]:
TAXA_DE_APRENDIZADO = 0.0005

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=11, out_features=150, bias=True)
    (1): ReLU()
    (2): Linear(in_features=150, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=80, bias=True)
    (5): ReLU()
    (6): Linear(in_features=80, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [12]:
NUM_EPOCAS = 2000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.3330)
1 tensor(0.3192)
2 tensor(0.3060)
3 tensor(0.2928)
4 tensor(0.2795)
5 tensor(0.2662)
6 tensor(0.2531)
7 tensor(0.2404)
8 tensor(0.2280)
9 tensor(0.2156)
10 tensor(0.2030)
11 tensor(0.1898)
12 tensor(0.1763)
13 tensor(0.1626)
14 tensor(0.1489)
15 tensor(0.1351)
16 tensor(0.1215)
17 tensor(0.1083)
18 tensor(0.0956)
19 tensor(0.0835)
20 tensor(0.0725)
21 tensor(0.0628)
22 tensor(0.0547)
23 tensor(0.0487)
24 tensor(0.0450)
25 tensor(0.0436)
26 tensor(0.0443)
27 tensor(0.0467)
28 tensor(0.0497)
29 tensor(0.0523)
30 tensor(0.0538)
31 tensor(0.0539)
32 tensor(0.0526)
33 tensor(0.0504)
34 tensor(0.0477)
35 tensor(0.0449)
36 tensor(0.0423)
37 tensor(0.0401)
38 tensor(0.0384)
39 tensor(0.0373)
40 tensor(0.0367)
41 tensor(0.0364)
42 tensor(0.0364)
43 tensor(0.0365)
44 tensor(0.0366)
45 tensor(0.0366)
46 tensor(0.0365)
47 tensor(0.0362)
48 tensor(0.0358)
49 tensor(0.0352)
50 tensor(0.0346)
51 tensor(0.0339)
52 tensor(0.0332)
53 tensor(0.0325)
54 tensor(0.0319)
55 tensor(0.0314)
56

In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.803664] [0.79874089]
[0.721097] [0.66635193]
[0.27111701] [0.24715702]
[0.84661497] [0.8732483]
[0.77047001] [0.74224967]
[0.223383] [0.23839471]
[0.68672899] [0.68552616]
[0.78892098] [0.73091292]
[0.53113902] [0.52868222]
[0.84562299] [0.87325255]
[0.87955599] [0.85161384]
[0.78057098] [0.75107151]
[0.30288899] [0.2625732]
[0.86152498] [0.83946901]
[0.82001001] [0.84727663]
[0.66264] [0.64278094]
[0.42030401] [0.63014786]
[0.89789798] [0.85655343]
[0.77956401] [0.6951647]
[0.52906397] [0.51189446]
[0.32713399] [0.31017426]
[0.85649399] [0.8416322]
[0.311915] [0.31017426]
[0.52430498] [0.51363547]
[0.48495801] [0.63014786]
[0.811945] [0.78695045]
[0.69132702] [0.68959414]
[0.725891] [0.70093471]
[0.692027] [0.72614475]
[0.72611201] [0.68959414]
[0.50893901] [0.54794066]
[0.62359101] [0.58677964]
[0.83282197] [0.80491372]
[0.737951] [0.68206941]
[0.65692202] [0.66824395]
[0.84609802] [0.80325372]
[0.52490598] [0.55271514]
[0.80258703] [0.76469049]
[0.725514] [0.69393882]
[0.230074] 

In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=11, out_features=150, bias=True)
    (1): ReLU()
    (2): Linear(in_features=150, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=80, bias=True)
    (5): ReLU()
    (6): Linear(in_features=80, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[0.81500402] [0.84741935]
[0.54636399] [0.54268855]
[0.320884] [0.27425358]
[0.770908] [0.78288296]
[0.337309] [0.31641292]
[0.85877899] [0.84930606]
[0.706279] [0.67845424]
[0.696503] [0.68552616]
[0.34748901] [0.33113609]
[0.84756002] [0.83505527]
[0.70656499] [0.68813585]
[0.87377498] [0.8820348]
[0.55245498] [0.59198741]
[0.41821299] [0.36715169]
[0.68829798] [0.72171525]
[0.666691] [0.68435734]
[0.71481701] [0.68195965]
[0.88304901] [0.86825584]
[0.89688597] [0.89632639]
[0.59309502] [0.68752529]
[0.36248199] [0.34768697]
[0.71725401] [0.72344105]
[0.829012] [0.83466263]
[0.791027] [0.74234483]
[0.73646402] [0.70054991]
[0.267539] [0.32319673]
[0.74453998] [0.73663587]
[0.90191099] [0.68752529]
[0.76071502] [0.73826137]
[0.88085603] [0.76115517]
[0.87084098] [0.84023993]
[0.72560999] [0.72124419]
[0.272509] [0.25997631]
[0.56935001] [0.46901697]
[0.31756999] [0.28728265]
[0.216026] [0.25689119]
[0.81453501] [0.78301081]
[0.760401] [0.81163496]
[0.67104502] [0.66271841]
[0.48551499

In [16]:
X_teste[0]

tensor([0.0000, 1.0000, 0.0000, 0.0000, 0.0546, 0.2159, 0.0000, 0.0000, 0.0000,
        0.0000, 0.2344])

In [17]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 0.05478899595705653


## Algoritmo Genético

In [18]:
# Constantes de busca

TAMANHO_POP = 12 # quantidade de indivíduos
NUM_GERACOES = 2000 # número de gerações
CHANCE_CRUZAMENTO = 0.5 # chance de ocorrer o cruzamento entre dois indivíduos
CHANCE_MUTACAO = 0.02 # chance de ocorrer mutação em cada indivíduo durante cada geração

# Constantes de problema
QUANTIDADE_MAX_ATOMOS = 100 # quantidade de valor máximo que um gene pode assumir
NUM_ELEMENTOS = len(FEATURES) - 3 # quantidade de genes presentes em cada indivíduo
TAMANHO_PORO_DESEJADO = 0.812

In [19]:
# Funções Locais

def cria_populacao_inicial(tamanho, numero_elementos):
    return populacao_mof(tamanho, numero_elementos, QUANTIDADE_MAX_ATOMOS)

def funcao_mutacao(individuo):
    return mutacao_mof(individuo, QUANTIDADE_MAX_ATOMOS)

def lista_para_dataframe(lista):
    """ Uma função que recebe uma lista e retorna um dataframe, seguindo uma ordem de colunas
    
    Args:
        lista: uma lista qualquer
        
    Return:
        dataframe
    """
    df_lista = pd.DataFrame([lista], columns=FEATURES)
    
    return df_lista

def computa_void_fraction(individuo):
    """ Uma função que calcula o tamanho previsto para uma MOF a partir do modelo de Redes Neurais treinado.
    
    Args:
        individuo: um indivíduo válido para o problema das mofs
        
    Return:
        void_fraction previsto
    """
    individuo = lista_para_dataframe(individuo)
    individuo = normalizador_x.transform(individuo)
    individuo = torch.tensor(individuo, dtype=torch.float32)
    with torch.no_grad():
        y_pred = minha_MLP(individuo)
        y_pred = normalizador_y.inverse_transform(y_pred)
        
    return y_pred

def funcao_objetivo(individuo, tamanho_poro_desejado):
    """ Uma função que calcula o fitness de cada indivíduo do problema das mofs a partir do modelo preditivo de redes neurais
    
    Args:
        individuo: um indivíduo válido para o problema das mofs
        
    Return:
        O fitness do indivíduo
    """
    y_pred = computa_void_fraction(individuo)
    
    return abs(y_pred - tamanho_poro_desejado)

def funcao_objetivo_pop(populacao):
    """ Calcula a função objetivo para todos os membros de uma população
    
    Args:
        população: Lista com todos os indivíduos da população
        
    Return:
        Lista contendo o fitness de cada indivíduo
    """
    fitness = []
    for individuo in populacao:
        fobj = funcao_objetivo(individuo, TAMANHO_PORO_DESEJADO)
        fitness.append(fobj)
    return fitness

In [20]:
populacao = cria_populacao_inicial(TAMANHO_POP, NUM_ELEMENTOS) # cria aleatoriamente uma população inicial

print('População inicial:') # mostra qual foi a população criada aleatoriamente
for i, ind in enumerate(populacao):
    print('Individuo ', i+1, ': ', ind)

for _ in range(NUM_GERACOES): # loop que começa a rodar cada geração
    fitness = funcao_objetivo_pop(populacao) # cálculo da função objetivo de cada indivíduo da população
    populacao = funcao_selecao(populacao, fitness) # seleção de roleta com diferentes pesos, baseados na função fitness
    
    pais = populacao[0::2] # definição dos indivíduos que serão pais
    maes = populacao[1::2] # definição dos indivíduos que serão mães
    contador = 0 # estratégia para colocar os filhos no lugar dos pais
    for pai, mae in zip(pais, maes): # laço de repetição para pegar itens da lista de pais e mães
        if random.random() < CHANCE_CRUZAMENTO: # aplicando a possibilidade de cruzamento
            # vai acertar o cruzamento
            filho1, filho2 = funcao_cruzamento(pai, mae) # "calculando" o filho 1 e o filho 2
            populacao[contador] = filho1 # trocando o pai pelo filho 1
            populacao[contador + 1] = filho2 # trocando a mãe pelo filho 2
            
        contador = contador + 2 # atualização do contador
    
    for n in range(len(populacao)): #laço de repetição para mutação
        if random.random() <= CHANCE_MUTACAO: # chance de mutação
            individuo = populacao[n] # esxolhe o indivíduo
            populacao[n] = funcao_mutacao(individuo) # muta o indivíduo
        
    
print()
print('População final:') # mostra qual foi a população final selecionada geneticamente
for i, ind in enumerate(populacao):
    print('Individuo ', i+1, ': ', ind, computa_void_fraction(ind))

População inicial:
Individuo  1 :  [180, 5, 7, 50, 74, 73, 80, 91, 24, 5, 5]
Individuo  2 :  [149, 2, 4, 74, 96, 64, 19, 94, 25, 44, 78]
Individuo  3 :  [83, 2, 8, 81, 75, 8, 94, 12, 69, 16, 89]
Individuo  4 :  [70, 1, 2, 57, 52, 53, 80, 38, 97, 19, 19]
Individuo  5 :  [66, 7, 12, 43, 84, 86, 66, 30, 31, 22, 28]
Individuo  6 :  [199, 1, 1, 78, 7, 22, 75, 100, 27, 41, 90]
Individuo  7 :  [141, 2, 12, 35, 57, 83, 45, 24, 4, 83, 81]
Individuo  8 :  [171, 2, 5, 31, 93, 19, 62, 18, 14, 80, 17]
Individuo  9 :  [113, 4, 1, 22, 59, 17, 17, 0, 82, 89, 35]
Individuo  10 :  [6, 3, 10, 66, 76, 21, 25, 32, 14, 42, 61]
Individuo  11 :  [78, 1, 7, 29, 58, 39, 9, 0, 57, 63, 80]
Individuo  12 :  [78, 7, 6, 64, 34, 56, 91, 76, 83, 2, 63]

População final:
Individuo  1 :  [202, 5, 7, 18, 99, 95, 4, 0, 4, 79, 77] [[0.81119365]]
Individuo  2 :  [202, 5, 7, 18, 99, 95, 4, 0, 4, 79, 77] [[0.81119365]]
Individuo  3 :  [202, 5, 7, 18, 99, 95, 4, 0, 4, 79, 77] [[0.81119365]]
Individuo  4 :  [202, 5, 7, 18, 99, 